<a href="https://colab.research.google.com/github/SymonMuchemi/0x01-python-if_else_loops_functions/blob/main/mammals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Function to load images and labels from the dataset
def load_dataset(dataset_path):
    images = []
    labels = []
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (64, 64))  # Resize image to a fixed size
            images.append(image)
            labels.append(folder)
    return np.array(images), np.array(labels)


In [3]:
!unzip -qo mammals_dataset.zip

In [4]:
!dir

mammals  mammals_dataset.zip  sample_data


In [5]:
!ls -la mammals/

total 8
drwxrwxrwx 2 root root 4096 Feb 13 21:50 .
drwxr-xr-x 1 root root 4096 Mar 25 09:09 ..


In [ ]:
dataset_path = "./content/mammals/"
images, labels = load_dataset(dataset_path)

FileNotFoundError: [Errno 2] No such file or directory: './content/mammals/'

In [ ]:
# Convert labels to one-hot encoding
label_to_index = {label: i for i, label in enumerate(np.unique(labels))}
index_to_label = {i: label for label, i in label_to_index.items()}
labels = np.array([label_to_index[label] for label in labels])

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_to_index), activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
559/559 [==============================] - 12s 13ms/step - loss: 0.6595 - accuracy: 0.5943 - val_loss: 0.5895 - val_accuracy: 0.6939
Epoch 2/10
559/559 [==============================] - 8s 14ms/step - loss: 0.5556 - accuracy: 0.7181 - val_loss: 0.5213 - val_accuracy: 0.7367
Epoch 3/10
559/559 [==============================] - 4s 6ms/step - loss: 0.4848 - accuracy: 0.7676 - val_loss: 0.4911 - val_accuracy: 0.7774
Epoch 4/10
559/559 [==============================] - 4s 6ms/step - loss: 0.4358 - accuracy: 0.7936 - val_loss: 0.4528 - val_accuracy: 0.7920
Epoch 5/10
559/559 [==============================] - 4s 7ms/step - loss: 0.3942 - accuracy: 0.8206 - val_loss: 0.5142 - val_accuracy: 0.7739
Epoch 6/10
559/559 [==============================] - 4s 6ms/step - loss: 0.3523 - accuracy: 0.8424 - val_loss: 0.4308 - val_accuracy: 0.8056
Epoch 7/10
559/559 [==============================] - 3s 6ms/step - loss: 0.3102 - accuracy: 0.8638 - val_loss: 0.4699 - val_accuracy: 0.7941
Epo

In [ ]:
# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)
report = classification_report(y_test, y_pred, target_names=[index_to_label[i] for i in range(len(label_to_index))])
print(report)

156/156 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

         cat       0.82      0.79      0.81      2521
         dog       0.80      0.82      0.81      2444

    accuracy                           0.81      4965
   macro avg       0.81      0.81      0.81      4965
weighted avg       0.81      0.81      0.81      4965

